In [1]:
pip install owlready2

     |████████████████████████████████| 23.7 MB 1.8 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.34-cp37-cp37m-linux_x86_64.whl size=20419503 sha256=e1aefdddbccc6028bd4f386e2e43da0c326f00b228c6596a1960ae60abb37338
  Stored in directory: /root/.cache/pip/wheels/ec/bd/94/4b682aa2b2c7a9d3cd5c7e036f7267940a92ef55889de68db6
Successfully built owlready2


# Example of a small program that imports Gene Ontology, declares a rendering function, and finally displays each GO concept with its parent classes

In [2]:
from owlready2 import *
go = get_ontology("http://purl.obolibrary.org/obo/go.owl").load()
obo = go.get_namespace("http://purl.obolibrary.org/obo/")
print(obo.GO_0006310)
# prints obo.GO_0006310
def render(entity):
  label = entity.label.first()
  if label: return "%s:’%s’" % (entity.name, label)
  return entity.name
set_render_func(render)
print(obo.GO_0006310)
# prints GO_0006310:’DNA recombination’
for go_concept in go.classes():
  print(go_concept)
  for parent in go_concept.is_a:
    print(" is a %s" % parent)
# prints:
# GO_0000001:’mitochondrion inheritance’
# is a GO_0048308:’organelle inheritance’
# is a GO_0048311:’mitochondrion distribution’
# GO_0000002:’mitochondrial genome maintenance’
# is a GO_0007005:’mitochondrion organization’
# ...

Streaming output truncated to the last 5000 lines.
 is a Thing
GO_1990480:’obsolete geranyl diphosphate synthase’
 is a Thing
GO_1990481:’mRNA pseudouridine synthesis’
 is a GO_0001522:’pseudouridine synthesis’
 is a GO_0016556:’mRNA modification’
GO_1990482:’sphingolipid alpha-glucuronosyltransferase activity’
 is a GO_0008194:’UDP-glycosyltransferase activity’
GO_1990483:’Clr6 histone deacetylase complex I''’
 is a GO_0000118:’histone deacetylase complex’
 is a BFO_0000050:’part of’.some(GO_0000785:’chromatin’)
GO_1990484:’aerobic lactate catabolic process’
 is a GO_1903457:’lactate catabolic process’
GO_1990485:’anaerobic lactate catabolic process’
 is a GO_1903457:’lactate catabolic process’
GO_1990486:’anaerobic fatty acid catabolic process’
 is a GO_0009062:’fatty acid catabolic process’
GO_1990487:’anaerobic lignin catabolic process’
 is a GO_0046274:’lignin catabolic process’
GO_1990488:’anaerobic cellulose catabolic process’
 is a GO_0030245:’cellulose catabolic process’
GO_19

# Example of a small program for auditing Gene Ontology(OG). The program imports GO, defines the subparts() class method, and then searches for all cellular components related to “nucleus” using a linguistic approach and a semantic approach. Finally, it compares the results of both approaches.

In [6]:
from owlready2 import *
go = get_ontology("http://purl.obolibrary.org/obo/go.owl").load()
obo = go.get_namespace("http://purl.obolibrary.org/obo/")
  
  
with obo:
  class GO_0005575(Thing): # Redefines Cellular_component class
    @classmethod
    def subparts(self):
      results = []
      for construct in self.constructs():
        if isinstance(construct, Restriction) and (construct.property is obo.BFO_0000050):
          results.extend(construct.subclasses())
      return results



linguistic = set()
for x in go.search(label = "*nucleus*"):
  if issubclass(x, obo.GO_0005575):
    linguistic.add(x)
    
    
semantic = set()
def found(x):
  if not x in semantic:
    semantic.add(x)
    for y in x.subclasses(): found(y)
    for y in x.subparts(): found(y)
found(obo.GO_0005634)
  

print(linguistic - semantic)

# prints { GO_0071561:’nucleus-vacuole junction’,
# GO_0042025:’host cell nucleus’ }


{GO_0042025:’host cell nucleus’, GO_0071561:’nucleus-vacuole junction’}
